In [1]:
import warnings

warnings.filterwarnings("ignore")

import json
import math
import os
import re
import sys
from collections import Counter
from copy import deepcopy
from functools import reduce
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from rich import print
from tqdm import tqdm

In [2]:
def plot_mtx(mtx):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.matshow(mtx, cmap=plt.cm.Blues)
    for i in range(mtx.shape[0]):
        for j in range(mtx.shape[1]):
            c = mtx[i, j]
    plt.plot()

In [3]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """Convert a list of string to a list of other types

    pattern: a list of types for one item.
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [4]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

# head = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=True, sep=",", start_row=0, end_row=1
# )

# print(f"Head:\n{head}")
# print(f"First line: {head[0]}")
# print(f"Last line: {head[-1]}")

data = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=False, sep=" ", start_row=None, end_row=None
)


print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: Sensor at x=489739, y=1144461: closest beacon is at x=-46516, y=554951 | Last 
line: Sensor at x=3343176, y=75114: closest beacon is at x=2646619, y=229757

------

Convert to:

First line: ['Sensor', 'at', 'x=489739,', 'y=1144461:', 'closest', 'beacon', 'is', 'at', 
'x=-46516,', 'y=554951']

Last line: ['Sensor', 'at', 'x=3343176,', 'y=75114:', 'closest', 'beacon', 'is', 'at', 
'x=2646619,', 'y=229757']

------

In [5]:
%%time
def get_int(s):
    return int(s.split(",")[0].split(":")[0].split("=")[1])


from scipy.spatial.distance import cityblock

info_li = []
for line in data:
    s_p = [get_int(line[2]), get_int(line[3])]
    b_p = [get_int(line[8]), get_int(line[9])]
    dist = cityblock(s_p, b_p)
    info_li.append([s_p, b_p, dist])


def check_intersect(info, dest_y):
    y_dist = abs(info[0][1] - dest_y)
    if y_dist >= info[2]:
        return [None, None]
    else:
        x_dist = info[2] - y_dist
        return [info[0][0] - x_dist, info[0][0] + x_dist]


target_row = 2000000
intersect_li = [check_intersect(info, target_row) for info in info_li]

intersect_li = [li for li in intersect_li if li[0]]

from functools import cmp_to_key


def mycmp(a, b):
    if a[0] > b[0]:
        return 1
    elif a[0] < b[0]:
        return -1
    else:
        return 0


intersect_li.sort(key=cmp_to_key(mycmp))

count = intersect_li[0][1] - intersect_li[0][0] + 1
current_max = intersect_li[0][1]
for li in intersect_li[1:]:
    if li[0] > current_max:
        count += li[1] - li[0] + 1
        current_max = li[1]
    elif li[0] == current_max:
        count += li[1] - li[0]
        current_max = li[1]
    else:
        if li[1] > current_max:
            count += li[1] - current_max
            current_max = li[1]

p2remove = [info[1][0] for info in info_li if info[1][1] == target_row]

print(f"Answer to Q1: {count-len(set(p2remove))}")

Answer to Q1: 5878678

Wall time: 2.33 ms


In [6]:
%%time

right_li = []
left_li = []
for li in info_li:
    x, y = li[0][0], li[0][1]
    d = li[2]
    right_li.extend([x + y + d, x + y - d])
    left_li.extend([x - y + d, x - y - d])

left_li.sort()
right_li.sort()

for idx in range(len(left_li) - 1):
    if left_li[idx + 1] - left_li[idx] == 2:
        l = left_li[idx] + 1
    if right_li[idx + 1] - right_li[idx] == 2:
        r = right_li[idx] + 1

x = int((l + r) // 2)
y = int((r - l) // 2)

print(f"Answer to Q2: {4000000*x+y}")

Answer to Q2: 11796491041245

Wall time: 5.01 ms
